# Urban Green Space Analysis and Optimization Tool

In [2]:
!pip install osmnx geopandas folium matplotlib scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install osmnx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [4]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [5]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [6]:
from shapely.geometry import Polygon

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [9]:
# Function to fetch urban boundary and green area data
def fetch_urban_data(city_name):
    import osmnx as ox
    print(f"Fetching data for {city_name}...")
    
    # Fetch urban boundary
    boundary = ox.geocode_to_gdf(city_name)
    
    # Fetch all types of green areas suitable for planting trees
    green_tags = {
        "leisure": ["park", "garden"],
        "landuse": [ "grass", "meadow"],
        "natural": ["wood", "grassland"]
    }
    green_areas = ox.features_from_place(city_name, tags=green_tags)
    
    return boundary, green_areas

# Fetch data for a city(Mysuru)
city_name = "Mandya, India"
boundary, green_areas = fetch_urban_data(city_name)

print(boundary.head())
print(green_areas.head())


Fetching data for Mandya, India...
                                            geometry  bbox_west  bbox_south  \
0  MULTIPOLYGON (((76.32956 12.80421, 76.32963 12...  76.329565   12.217912   

   bbox_east  bbox_north   place_id  osm_type   osm_id        lat        lon  \
0  77.333804   13.055335  231810137  relation  2020591  12.636894  76.727548   

      class            type  place_rank  importance     addresstype    name  \
0  boundary  administrative          10    0.482136  state_district  Mandya   

               display_name  
0  Mandya, Karnataka, India  
                                                              geometry  \
element  id                                                              
node     410803782                           POINT (76.57288 12.42139)   
         1642087788                           POINT (76.57367 12.4274)   
         4641398514                          POINT (76.38116 12.76175)   
         6779117311                          POINT (76.8

In [10]:
# Function to calculate green space coverage
def analyze_green_space(boundary, green_areas):
    green_areas['area_sqm'] = green_areas.geometry.area
    total_green_space = green_areas['area_sqm'].sum()
    boundary_area = boundary.geometry.area.iloc[0]
    coverage_percentage = (total_green_space / boundary_area) * 100
    
    print(f"Total Green Space: {total_green_space:.2f} sq.m")
    print(f"City Area: {boundary_area:.2f} sq.m")
    print(f"Green Space Coverage: {coverage_percentage:.2f}%")
    
    return total_green_space, coverage_percentage

# Analyze green space for the city
total_green_space, coverage_percentage = analyze_green_space(boundary, green_areas)


Total Green Space: 0.04 sq.m
City Area: 0.41 sq.m
Green Space Coverage: 10.00%


In [11]:
# Function to predict environmental impact based on green space area
def predict_environmental_impact(green_space_areas, temperature_reductions):
    model = LinearRegression()
    green_space_areas = green_space_areas.values.reshape(-1, 1)
    temperature_reductions = temperature_reductions.values
    model.fit(green_space_areas, temperature_reductions)
    
    # Predict impact for 1000 sq.m of green space
    predicted_reduction = model.predict([[1000]])[0]
    print(f"Predicted Temperature Reduction for 1000 sq.m Green Space: {predicted_reduction:.2f}°C")
    return model

# Example data for prediction
green_space_data = pd.Series([100, 500, 1000, 1500, 2000])  # Green space areas in sq.m
temp_reduction_data = pd.Series([0.1, 0.5, 1.0, 1.5, 2.0])  # Corresponding temperature reductions in °C

# Train the model and predict
model = predict_environmental_impact(green_space_data, temp_reduction_data)


Predicted Temperature Reduction for 1000 sq.m Green Space: 1.00°C


In [12]:
from shapely.geometry import Polygon, MultiPolygon

# Function to create a map visualization
def visualize_data(boundary, green_areas, city_name):
    # Create a Folium map centered on the city's boundary centroid
    m = folium.Map(
        location=[boundary.geometry.centroid.y.iloc[0], boundary.geometry.centroid.x.iloc[0]], 
        zoom_start=12
    )
    
    # Add green areas to the map
    for _, row in green_areas.iterrows():
        geometry = row['geometry']
        
        # Check if the geometry is a Polygon
        if isinstance(geometry, Polygon):
            folium.Polygon(
                locations=[(pt[1], pt[0]) for pt in geometry.exterior.coords],
                color='green',
                fill=True,
                fill_opacity=0.5
            ).add_to(m)
        
        # Check if the geometry is a MultiPolygon
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:  # Use .geoms to iterate over individual polygons
                folium.Polygon(
                    locations=[(pt[1], pt[0]) for pt in polygon.exterior.coords],
                    color='green',
                    fill=True,
                    fill_opacity=0.5
                ).add_to(m)
    
    # Save the map as an HTML file
    map_file = f"{city_name.replace(' ', '_')}_green_space_map.html"
    m.save(map_file)
    print(f"Map saved as {map_file}")
    return m

# Visualize the city's green spaces
map_result = visualize_data(boundary, green_areas, city_name)


Map saved as Mandya,_India_green_space_map.html


In [13]:
# Display the map within Jupyter Notebook
from IPython.display import IFrame

map_file = f"{city_name.replace(' ', '_')}_green_space_map.html"
IFrame(map_file, width=800, height=600)
